# 感情分析に基づく分類

## パッケージのインストール

In [ ]:
!pip install oseti

## 基本的な使い方

In [2]:
import oseti

analyzer = oseti.Analyzer()
print(analyzer.analyze_detail('天国で待ってる。'))
print(analyzer.analyze_detail('遅刻したけど楽しかったし嬉しかった。すごく充実した！'))

[{'positive': ['天国'], 'negative': [], 'score': 1.0}]
[{'positive': ['楽しい', '嬉しい'], 'negative': ['遅刻'], 'score': 0.3333333333333333}, {'positive': ['充実'], 'negative': [], 'score': 1.0}]


In [3]:
from collections import Counter
import oseti

filename = 'text/kageotoko.corpus.txt'
analyzer = oseti.Analyzer()

with open(filename, 'r', encoding='UTF-8') as f:
    hist = Counter(
        element['score']
        for line in f
        for element in analyzer.analyze_detail(line))

print('score count')
print('-----------')
for k in sorted(hist.keys()):
    print('{:>5.02f} {}'.format(k, hist[k]))

score count
-----------
-1.00 989
-0.60 4
-0.50 14
-0.33 65
-0.20 7
-0.14 3
 0.00 3364
 0.14 1
 0.20 7
 0.33 68
 0.50 12
 0.60 5
 0.67 1
 1.00 986


# トピックモデルによるクラスタリング

## パッケージのインストール

In [ ]:
!pip install gensim pandas openpyxl

## 解析対象データをダウンロード

In [ ]:
!mkdir -p sisyou_db

for year in ['h18', 'h19', 'h20', 'h21', 'h22', 'h23', 'h24']:
    for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        !wget -P sisyou_db -c https://anzeninfo.mhlw.go.jp/anzen/shisyo_xls/sisyou_db_{year}_{month}.xls

for year in ['h25', 'h26', 'h27', 'h28', 'h29']:
    for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        !wget -P sisyou_db -c https://anzeninfo.mhlw.go.jp/anzen/shisyo_xls/sisyou_db_{year}_{month}.xlsx

## サンプルコードでは平成29年1月分のデータしか利用しませんが、
## データとしては平成18年〜平成29年の12年分のデータをダウンロードします。
## ぜひご自身でも分析してみてください、

## 基本的な使い方

In [6]:
import pandas
df=pandas.read_excel('sisyou_db/sisyou_db_h29_01.xlsx')
print(df['災害状況'])

0                                                     NaN
1       工場内で床に置いていたコードに、荷物を抱えていた状態のときに足が引っ掛かり、よろめいて数歩前...
2       倉庫の出入口の階段を荷物（冷凍商品15kgぐらい）を持って下りる際に、階段が凍っていて滑って...
3       会社構内にて車輌の洗車中、足を滑らせ転倒した際に左手をつき、翌朝に左肩の痛みが大きくなり、左...
4                       厩舎2階でバッカン受け入れ作業中、バッカンを落とす穴から落下した。
                              ...                        
2691    重機の整備中、待機している台船へ乗船時に、つまずいて高さ1m40㎝～50㎝の所から転落し、足...
2692    新聞配達中、アパートにて2階と3階の配達を終え、1階に下りる時に誤って最後の段で足を滑らせて...
2693    左手にしびれを感じ、中指にも痛みが出始めたたため検査した結果、手根管症候群と中指ばね指と診断...
2694    塗装場所へ移動する為、5尺の脚立をはしご状態にして、約2.3m程上がった屋上へ上る途中に使用...
2695    入浴介助後、利用者（男性48㎏・全介助・車いす）を洗い場から車いすに移動させる際、新人職員が...
Name: 災害状況, Length: 2696, dtype: object


In [7]:
import logging
import spacy
import pandas as pd 

# 1. 学習中の状況を出力するフォーマットの指定
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.INFO)

# 2. 解析器の初期化
nlp = spacy.load("ja_ginza")

# 3. ドキュメントを読み込んで単語分割する
df=pd.read_excel('sisyou_db/sisyou_db_h29_01.xlsx')
docs=[]
for text in df['災害状況']:
    if type(text) != str: continue
    doc=[token.lemma_ for token in nlp(text)]
    docs.append(doc)

#
# 4. 極端に頻度が低い語や高い語を除外する
#
from gensim.corpora import Dictionary

# 4a. 辞書の作成
dictionary = Dictionary(docs)

# 4b. 出現が20文書に満たない単語と50%以上の文書に出現する単語を極端とみなして除外する
dictionary.filter_extremes(no_below=20, no_above=0.5)

# 4c. 上記のように定められた語彙で、文書単語行列(Bag-of-words表現)を求める
corpus = [dictionary.doc2bow(doc) for doc in docs]

print(f'Number of unique tokens: {len(dictionary)}')
print(f'Number of documents: {len(corpus)}')

#
# 5. バイグラムの計算
#
from gensim.models import Phrases

# 5a. 文書単語行列にバイグラムを加える (出現頻度20以上のもの)
phrase_model = Phrases(docs, min_count=20)
for idx, doc in enumerate(docs):
    # _が含まれるトークンはバイグラム(あるいはそれ以上)なので、追加する
    bigram_tokens = [token for token in phrase_model[doc] if '_' in token]
    docs[idx].extend(bigram_tokens)

#            
# 6. LDAモデル
#
from gensim.models import LdaModel

num_topics = 10

temp = dictionary[0]  # 辞書をメモリに読み込むための処理

# 6a. LDAモデルの計算
model = LdaModel(
    corpus=corpus,
    id2word=dictionary.id2token,
    chunksize=2000,
    alpha='auto',
    eta='auto',
    iterations=400,
    num_topics=num_topics,
    passes=20,
)

# 6b. LDAモデルによって得られるトピックの抽出
top_topics = model.top_topics(corpus) 
coherences = [coherence for topic_words, coherence in top_topics]

# 6c. 評価指標コヒーレンスを求める
avg_topic_coherence = sum(coherences) / num_topics
print(f'Average topic coherence: {avg_topic_coherence:.4f}.')

2023-04-19 22:02:37,057 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-04-19 22:02:37,157 : INFO : built Dictionary<7284 unique tokens: ['、', '。', 'ある', 'いる', 'が']...> from 2695 documents (total 148921 corpus positions)
2023-04-19 22:02:37,158 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<7284 unique tokens: ['、', '。', 'ある', 'いる', 'が']...> from 2695 documents (total 148921 corpus positions)", 'datetime': '2023-04-19T22:02:37.158527', 'gensim': '4.3.1', 'python': '3.9.10 (main, Mar  9 2022, 00:27:09) \n[Clang 13.0.0 (clang-1300.0.27.3)]', 'platform': 'macOS-12.2.1-arm64-arm-64bit', 'event': 'created'}
2023-04-19 22:02:37,162 : INFO : discarding 6668 tokens: [('、', 2673), ('。', 2689), ('いる', 1436), ('が', 1856), ('する', 2506), ('た', 2661), ('て', 2264), ('で', 1680), ('に', 2504), ('の', 2328)]...
2023-04-19 22:02:37,163 : INFO : keeping 616 tokens which were in no less than 20 and no more than 1347 (=50.0%) documents
2023-04-19 22:02:37,164 : INFO : result

Number of unique tokens: 616
Number of documents: 2695


2023-04-19 22:02:37,412 : INFO : collected 46390 token types (unigram + bigrams) from a corpus of 148921 words and 2695 sentences
2023-04-19 22:02:37,413 : INFO : merged Phrases<46390 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000>
2023-04-19 22:02:37,414 : INFO : Phrases lifecycle event {'msg': 'built Phrases<46390 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000> in 0.21s', 'datetime': '2023-04-19T22:02:37.414762', 'gensim': '4.3.1', 'python': '3.9.10 (main, Mar  9 2022, 00:27:09) \n[Clang 13.0.0 (clang-1300.0.27.3)]', 'platform': 'macOS-12.2.1-arm64-arm-64bit', 'event': 'created'}
2023-04-19 22:02:37,583 : INFO : using autotuned alpha, starting with [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
2023-04-19 22:02:37,584 : INFO : using serial LDA version on this node
2023-04-19 22:02:37,585 : INFO : running online (multi-pass) LDA training, 10 topics, 20 passes over the supplied corpus of 2695 documents, updating model once every 2000 documents, evaluatin

Average topic coherence: -2.2068.
